## Clustering via K-Medoids

In Notebook 14, you implemented clustering via k-means. In this problem, you will learn "k-medoids". Both the k-means and k-medoids algorithms are partitional (breaking the dataset up into groups). K-means attempts to minimize the total squared error from a central position in each cluster (centroids), while k-medoids minimizes the sum of dissimilarities between objects labeled to be in a cluster and one of the objects designated as the representative of that cluster (medoids). In contrast to the k-means algorithm, k-medoids chooses medoids from the objects in the set, instead of a central, average position in the feature space.

The possible choice of the dissimilarity function is very rich but in our problem, we will use the more general, __Minkowski Distance Function__ of a given order. (The formula will be provided to you in the exercises)

The algorithm that we will be implementing is called the __PAM algorithm__ (Partitioning Around Medoids, (Kaufman and Rousseeuw 1990)). It can be summarized as follows:
1. __Initialize__: randomly select $k$ of the $m$ data points as the medoids
2. __Assignment step__: Associate each data point to the closest medoid.
3. __Update step__: For each medoid $j$ and each data point $i$ associated to $j$ swap $j$ and $i$ and compute the total cost of the configuration (that is, the average dissimilarity of $i$ to all the data points associated to $j$). Select the medoid $j$ with the lowest cost of the configuration.
Repeat alternating steps 2 and 3 until there is __no change__ in the assignments.

As we did in the notebooks, in the code that follows, it will be convenient to use our usual "data matrix" convention, that is, in the data matrix $X$, each row, $\hat{x}$, is one of  $m$  observations and each column, $x$, is one of $f$  features. However, we will not need a dummy column of ones since we are not fitting a function.


\begin{equation*}
X \equiv
\begin{bmatrix}
x_{0,0} \cdots x_{0,f-1} \\
\vdots\\
x_{m-1,0} \cdots x_{m,f-1}\\
\end{bmatrix}
\end{equation*}

We will use a subset of https://archive.ics.uci.edu/ml/datasets/seeds#, `seeds_subset.txt`.

It is a comma-separated-value file with a header row and four columns and 210 rows. The four rows are the Area, Asymmetry, Groove Length, and Type of seed. The Type column is the "true" classification/clustering.

In [1]:
# Imports
import pandas as pd
import numpy as np
import os

WD = "../resource/asnlib/publicdata/"
DATA_PATH = WD + 'seeds_subset.txt'
df = pd.read_csv(DATA_PATH)
df.head()

FileNotFoundError: [Errno 2] File b'../resource/asnlib/publicdata/seeds_subset.txt' does not exist: b'../resource/asnlib/publicdata/seeds_subset.txt'

Here's a visualization of the data, using Plotly Python.

The figure will choose the hue for each point using its `Type`, which is the true assignment of the cluster.

`Scatter3D` Plotly figures allow for rotation of the figure using your mouse, which can clarify the relationships between clusters and features.

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

trace1 = go.Scatter3d(
    x=df['Asymmetry'],
    y=df['Groove'],
    z=df['Area'],
    mode='markers',
    marker=dict(
        size=6,
        color=df['Type'],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    scene = dict(
        xaxis = dict(
            title='Asymmetry'),
        yaxis = dict(
            title='Groove'),
        zaxis = dict(
            title='Area'),),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    showlegend=False
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

As you can see, there are some points that intermingle between the clusters, so there won't be a perfect answer using k-medoids, which establishes "hard" hyperplanes between clusters and does not allow for mixing between clusters.

Let's extract the data points as a two-dimensional NumPy ndarray, points, and the labels as a one-dimensional NumPy ndarray, labels. Note that the k-medoids algorithm you will implement should not reference labels -- that's the solution we will try to predict given only the point coordinates (points) and target number of clusters (k).

In [ ]:
points = df[['Asymmetry', 'Groove', 'Area']].values
labels = df['Type'].values
m, f = points.shape
k = 3

If we now plot the `points` without the `labels`, we have the "unsupervised clustering problem" visualized. That is, with no prior knowledge of label assignment, generate `k` clusters. In fact, `k` won't even be known ahead of time, and with large dimensional data, may not be easily visualized.

In [ ]:
trace1 = go.Scatter3d(x=points[:,0],y=points[:,1],z=points[:,2],mode='markers',marker=dict(size=6,colorscale='Viridis',opacity=0.8))
layout = go.Layout(scene = dict(xaxis = dict(title='Asymmetry'),yaxis = dict(title='Groove'),zaxis = dict(title='Area')),margin=dict(l=0,r=0,b=0,t=0),showlegend=False)
data = [trace1]; fig = go.Figure(data=data, layout=layout); iplot(fig)

### Medoid Initialization:
To start the algorithm, you need an initial guess. Let's randomly choose  k  observations from the data.

__Exercise 1__ (0.5 points) Complete the following function, init_medoids(X, k), so that it randomly selects  k  of the given observations to serve as medoids. It should return a NumPy array of size $k \times d$, where d is the number of columns of X. You may use modules and methods from NumPy or the Python Standard Library.

In [ ]:
def init_medoids(X, k):
    """
    Randomly samples k observations from X as medoids.
    Returns these medoids as a numpy array, with shape (k,f).
    """
    ###
    ### YOUR CODE HERE
    ###


In [ ]:
# Test cell: `init_medoids_test`

medoids_initial = init_medoids(points, k)
print(f"Initial medoids:\n {medoids_initial}")

assert type(medoids_initial) is np.ndarray, f"Your function should return a NumPy array instead of a {type(medoids_initial)}"
assert medoids_initial.shape == (k, f), f"Returned medoids do not have the right shape ({k}, {f})"
for i in range(f):
    assert medoids_initial[i,:] in points, "The medoids must come from the input."

print("\n(Passed!)")

The plot below shows the random initial choice of medoids in <font color=red>**red**</font>. Running it repeatedly will show the variety of combinations that can be chosen.

In [ ]:
plot_medoids_init = init_medoids(points, k)

trace1 = go.Scatter3d(x=points[:,0],y=points[:,1],z=points[:,2],mode='markers',marker=dict(size=6,colorscale='Viridis',opacity=0.5))
trace2 = go.Scatter3d(x=plot_medoids_init[:,0],y=plot_medoids_init[:,1],z=plot_medoids_init[:,2],mode='markers',marker=dict(size=6,color='red',opacity=1))
layout = go.Layout(scene = dict(xaxis = dict(title='Asymmetry'),yaxis = dict(title='Groove'),zaxis = dict(title='Area')),margin=dict(l=0,r=0,b=0,t=0),showlegend=False)
data = [trace1, trace2]; fig = go.Figure(data=data, layout=layout); iplot(fig)

### Computing the distances
https://en.wikipedia.org/wiki/Minkowski_distance
    
__Exercise 2__ (2 points). Implement a function that computes a distance matrix,  $S=(s_{ij})$  such that  $s_{ij}=d^{p}_{ij}$  is the $p^{th}$ power of the Minkowski distance (order $p$) from point  $x_i$  to medoid  $\mu_j$ . It should return a NumPy matrix S with shape (m,k).

The $p^{th}$ order Minkowski Distance between two points, $x$ and $\mu$ is given by:

$$\begin{equation*}
d_{ij} = \bigg(\sum^{f}_{a=1}\mid x_{ia} - \mu_{ja} \mid ^p\bigg)^{\frac{1}{p}} \textrm{,} \quad  0 < i < (m-1) \textrm{,} \quad  0 < j < (k-1)
\end{equation*}$$

Thus, for the elements of $S$,

$$\begin{equation*}
s_{ij} = d^{p}_{ij} = \sum^{f}_{a=1}\mid x_{ia} - \mu_{ja} \mid ^p
\end{equation*}$$

Minkowski distance is typically used with $p$ being 1 or 2, which correspond to the __Manhattan distance__ and the __Euclidean distance__, respectively, but $p$ is left as an input parameter in this function.


In [ ]:
def compute_d_p(X, medoids, p):
    m = len(X)
    medoids_shape = medoids.shape
    # If a 1-D array is provided, 
    # it will be reshaped to a single row 2-D array
    if len(medoids_shape) == 1: 
        medoids = medoids.reshape((1,len(medoids)))
    k = len(medoids)
    
    S = np.empty((m, k))
    
    ###
    ### YOUR CODE HERE
    ###

    return S

In [ ]:
# Test cell: `compute_d_p`
medoids = np.loadtxt(WD + 'test_medoids.txt')
S_test2 = np.loadtxt(WD + 'test2_soln.txt')
    
student_soln = compute_d_p(points, medoids, 2)
assert (np.linalg.norm (S_test2 - student_soln, axis=1) <= (100.0 * np.finfo(float).eps)).all (), "Your solution is not similar enough to the instructor's solution"
print("\n(Passed!)")

### Cluster Assignment
__Exercise 3__ (1 point): Write a function that acts on the $p^{th}$ order distance matrix to assign a "cluster label" to each point using the minimum distance to find the "most similar" medoid.

That is, consider the  $m \times k$,  $p^{th}$ power distance matrix  S .

For each point, indicated by row index $i$ , if  $s_{ij}$  is the minimum distance for point  $i$ , then the index  $j+1$  is  $i$ 's cluster label.

In other words, your function should return a one-dimensional array,  $y$,  of length  $m$  such that

\begin{equation*}
y_{i} = \underset{j∈\{0,…,k−1\}}{\operatorname{argmin}}s_{ij} + 1
\end{equation*}

In [ ]:
def assign_cluster_labels(S):
    ###
    ### YOUR CODE HERE
    ###

# Cluster labels:     1    2
S_test1 = np.array([[0.3, 0.2],  # --> cluster 2
                    [0.1, 0.5],  # --> cluster 1
                    [0.4, 0.2]]) # --> cluster 2
y_test1 = assign_cluster_labels(S_test1)
print("You found:", y_test1)

# Debugging assert
assert (y_test1 == np.array([2, 1, 2])).all()

In [ ]:
# Test cell: `assign_cluster_labels_test`


S_test2 = np.loadtxt(WD + 'test2_soln.txt')

student_soln = assign_cluster_labels(S_test2)
y_test2 = np.loadtxt(WD + 'test3_soln.txt')
assert (y_test2 == student_soln).all(), f"Your solution doesn't match the instructor's solution! \n First 10 cluster labels of the instructor's solution: \n {y_test2[:10]}"

print("\n(Passed!)")

The initial assignment to clusters, relative to the initial medoids (<font color=red>**red**</font>) is visualized below. 

The colors may not correlate to the original plot, since the medoids are randomly assigned, so is their row order, which determines the color.

By re-running the code cell below, you can see how the assignment is affected by the random choice of initial medoids.

In [ ]:
plot_medoids = init_medoids(points, k)
plot_y = assign_cluster_labels(compute_d_p(points, plot_medoids, p=2))

trace1 = go.Scatter3d(x=points[:,0],y=points[:,1],z=points[:,2],mode='markers',marker=dict(size=6,color=plot_y,colorscale='Viridis',opacity=0.5))
trace2 = go.Scatter3d(x=plot_medoids[:,0],y=plot_medoids[:,1],z=plot_medoids[:,2],mode='markers',marker=dict(size=6,color='red',opacity=1))
layout = go.Layout(scene = dict(xaxis = dict(title='Asymmetry'),yaxis = dict(title='Groove'),zaxis = dict(title='Area')),margin=dict(l=0,r=0,b=0,t=0),showlegend=False)
data = [trace1, trace2]; fig = go.Figure(data=data, layout=layout); iplot(fig)

### Swap Step

As mentioned in the introduction, in this step, for each medoid  $j$  and each data point  $i$  associated to  $j$  swap  $j$  and  $i$  and compute the total cost of the configuration (that is the average dissimilarity of  $i$  to all the data points, $\sum s_{ij}$). Select the medoid  $j$  with the lowest cost of the configuration.

That is, for each cluster, search if any of the points in the cluster decreases the average dissimilarity coefficient. Select the point that decreases this coefficient the most as the medoid for this cluster. 

Dissimilarity is given by __Minkowski(p)__, so you could use functions created in earlier exercises.

__Exercise 4__ (4 points): Given a clustering (i.e. a set of points and their medoids) and order of $p$, compute the new medoid of each cluster. 

__Note__: The computation of medoids is __NOT__ the same as that of cluster means. 

__Reference__: This link could be helpful in better understanding the swap step!
https://www.youtube.com/watch?v=OWpRBCrx5-M

In [ ]:
def update_medoids(X, medoids, p):
    
    ###
    ### YOUR CODE HERE
    ###
    
    return out_medoids

In [ ]:
# Test cell: `update_medoids_test`

medoids = np.loadtxt(WD + 'test_medoids.txt')

student_soln = update_medoids(points, medoids, 2)
medoids_test = np.loadtxt(WD + 'test4_soln.txt')
#medoids_test = np.loadtxt(WD + 'test4_soln_cook.txt')

for medoid in student_soln:
    assert medoid in points, "Your updated medoids are not points in the original data set!"
delta_test3 = np.abs(medoids_test - student_soln)
assert (delta_test3 <= 2.0*len(medoids_test)*np.finfo(float).eps).all(), "Your solution does not match the instructor's solution"
print("\n(Passed!)")

In the visualization below, you can see how the updated medoids (**black**) are moved relative to the initial medoids (<font color=red>**red**</font>).

If you don't see a red point, it's because the medoid didn't move! Just because they're chosen randomly doesn't mean they aren't at the minimum point!

In [ ]:
plot_medoids = init_medoids(points, k)
plot_y = assign_cluster_labels(compute_d_p(points, plot_medoids, p=2))
plot_medoids_updated = update_medoids(points, plot_medoids, 2)

trace1 = go.Scatter3d(x=points[:,0],y=points[:,1],z=points[:,2],mode='markers',marker=dict(size=6,color=plot_y,colorscale='Viridis',opacity=0.5))
trace2 = go.Scatter3d(x=plot_medoids[:,0],y=plot_medoids[:,1],z=plot_medoids[:,2],mode='markers',marker=dict(size=6,color='red',opacity=1))
trace3 = go.Scatter3d(x=plot_medoids_updated[:,0],y=plot_medoids_updated[:,1],z=plot_medoids_updated[:,2],mode='markers',marker=dict(size=6,color='black',opacity=1))
layout = go.Layout(scene = dict(xaxis = dict(title='Asymmetry'),yaxis = dict(title='Groove'),zaxis = dict(title='Area')),margin=dict(l=0,r=0,b=0,t=0),showlegend=False)
data = [trace1, trace2, trace3]; fig = go.Figure(data=data, layout=layout);iplot(fig)

__Exercise 5__(0.5 points): Lastly, write a function to check whether the medoids have "moved," given two instances of the medoid values. It accounts for the fact that the order of medoids may have changed.

In [ ]:
def has_converged(old_medoids, medoids):
    ###
    ### YOUR CODE HERE
    ###


In [ ]:
# Test cell: `has_converged`
assert has_converged(medoids_initial, medoids_initial) == True
print("\n(Passed!)")

__Exercise 6__ (2 points): Put all of the preceding building blocks together to implement the PAM k-medoids algorithm.

1. __Initialize__: randomly select $k$ of the $m$ data points as the medoids
2. __Assignment step__: Associate each data point to the closest medoid.
3. __Update step__: For each medoid $j$ and each data point $i$ associated to $j$ swap $j$ and $i$ and compute the total cost of the configuration (that is, the average dissimilarity of $i$ to all the data points associated to $j$). Select the medoid $j$ with the lowest cost of the configuration.
Repeat alternating steps 2 and 3 until there is __no change__ in the assignments.

In [ ]:
def kmedoids(X, k, p, starting_medoids=None, max_steps=np.inf):
    if starting_medoids is None:
        medoids = init_medoids(X, k)
    else:
        medoids = starting_medoids
        
    converged = False
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_medoids = medoids.copy()
        ###
        ### YOUR CODE HERE
        ###
        print(medoids)
        i += 1
    return labels

In [ ]:
# Test cell: `kmedoids_test`

# Helper Functions

def mark_matches(a, b, exact=False):
    """
    Given two Numpy arrays of {0, 1} labels, returns a new boolean
    array indicating at which locations the input arrays have the
    same label (i.e., the corresponding entry is True).
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as the same up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    assert a.shape == b.shape
    a_int = a.astype(dtype=int)
    b_int = b.astype(dtype=int)
    all_axes = tuple(range(len(a.shape)))
    assert ((a_int == 1) | (a_int == 2) | (a_int == 3)).all()
    assert ((b_int == 1) | (b_int == 2) | (b_int == 3)).all()
    
    exact_matches = (a_int == b_int)
    if exact:
        return exact_matches

    assert exact == False
    num_exact_matches = np.sum(exact_matches)
    if (2*num_exact_matches) >= np.prod (a.shape):
        return exact_matches
    return exact_matches == False # Invert

def count_matches(a, b, exact=False):
    """
    Given two sets of {0, 1} labels, returns the number of mismatches.
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as similar up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    matches = mark_matches(a, b, exact=exact)
    return np.sum(matches)

clustering = kmedoids(points, 3, 2, medoids)
df['clustering'] = clustering
n_matches = count_matches(df['Type'], df['clustering'])
print(n_matches,
      "matches out of",
      len(df), "possible",
      "(~ {:.1f}%)".format(100.0 * n_matches / len(df)))

assert n_matches >= 160
print("\n(Passed!)")

This plot shows the clustering results of your k-medoid implementation

In [ ]:
y_true = df['Type'].values
y_final_trans = np.ones(clustering.shape, 'int64')*-1
for i in range(1,4):
    y_final_trans[clustering == np.argmax(np.bincount(clustering[y_true==i]))] = i
trace = go.Scatter3d(x=df['Asymmetry'],y=df['Groove'],z=df['Area'],mode='markers',marker=dict(size=6,color=y_final_trans,colorscale='Viridis',opacity=.8))
layout = go.Layout(scene = dict(xaxis = dict(title='Asymmetry'),yaxis = dict(title='Groove'),zaxis = dict(title='Area')),margin=dict(l=0,r=0,b=0,t=0),showlegend=False)
data = [trace];fig = go.Figure(data=data, layout=layout);iplot(fig)

The final visualization is a comparison of the original clusters with the k-medoid output.

Gray points are matches between the original and your output.

Using color and opacity, the other colors indicate the various combinations of mismatches.

Not bad!

In [ ]:
y_true = df['Type'].values
y_final_trans = np.ones(clustering.shape, 'int64')*-1
for i in range(1,4):
    y_final_trans[clustering == np.argmax(np.bincount(clustering[y_true==i]))] = i
trace1 = go.Scatter3d(x=df['Asymmetry'],y=df['Groove'],z=df['Area'],mode='markers',marker=dict(size=6,color=df['Type'],colorscale=[[0.0,'rgb(255,255,0)'],[0.5,'rgb(0,255,255)'],[1.0,'rgb(255,0,255)']],opacity=.5))
trace2 = go.Scatter3d(x=df['Asymmetry'],y=df['Groove'],z=df['Area'],mode='markers',marker=dict(size=6,color=y_final_trans,colorscale=[[0.0,'rgb(0,0,127)'],[0.5,'rgb(127,0,0)'],[1.0,'rgb(0,127,0)']],opacity=.5))
layout = go.Layout(scene = dict(xaxis = dict(title='Asymmetry'),yaxis = dict(title='Groove'),zaxis = dict(title='Area')),margin=dict(l=0,r=0,b=0,t=0),showlegend=False)
data = [trace1, trace2];fig = go.Figure(data=data, layout=layout);iplot(fig)

__Fin!__ That marks the end of this problem. Don't forget to submit it!